In [16]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
os.environ["OPEN_AI_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [18]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("OPEN_AI_KEY"), api_version=os.getenv("OPENAI_API_VERSION"))


In [19]:
result = llm.invoke("What is Google")

print(result.content)

Google is a multinational technology company specializing in various Internet-related services and products. The company was founded by Larry Page and Sergey Brin while they were pursuing their Ph.D. degrees at Stanford University in California. Google was incorporated on September 4, 1998.

Google's core product is its search engine, which is the most widely used web-based search service globally, allowing users to find relevant information on the internet. Over the years, the company has expanded its range of services to include advertising technologies (like Google Ads), cloud computing, software, and hardware solutions.

Notable products and services offered by Google include:
- **Google Search**: The search engine that indexes websites and allows users to search for information online.
- **Android**: An operating system used on a majority of the world's smartphones and other devices.
- **Gmail**: An email service with a significant user base worldwide.
- **Google Maps**: A mapping

In [20]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# prompt=ChatPromptTemplate.from_messages([
#     ("system", "you are customer support from doko moko you need to asnwer user queries"),
#     ("user", "{input}")
# ])
# prompt

output_parser  = JsonOutputParser()
# prompt = PromptTemplate(
#     template="Answer the user query. \n{format_instructions}\n{query}",
#     input_variables=["query"],
#     partial_variables={"format_instructions":{output_parser.get_format_instructions()}}
# )

In [21]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/RajuGangitla/nueral-backend")
loader


In [22]:
document = loader.load()

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitting = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitting.split_documents(document)

In [24]:
import os
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("OPEN_AI_KEY"),
        api_version="2023-05-15"
        )

In [25]:
from langchain_community.vectorstores import FAISS

vectorstore  = FAISS.from_documents(documents, embeddings)
vectorstore

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the follwing questions from the provided context : 
    <content>
    {context}
    </content>
    """
)


In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


In [ ]:
document_chain.invoke({"input":"Who is Raju Gangitla "})
query = "Raju"
result = vectorstore.similarity_search(query=query)

result[0].page_content

KeyError: 'context'